In [3]:
# import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm

from imp import reload
import rl_dqn
import environment
import embedding

from rl_dqn import ReplayMemory, DQN, Transition
from embedding import EmbeddingModel
from environment import OriginAgent, DialougeEnv
from transformers import pipeline


/tmp/ipykernel_735/1876905766.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
2023-05-30 13:52:49.473732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-30 13:52:49.473771: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-30 13:52:52.735911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-30 13:52:52.735956: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-30 13:52:52.736016: I tensorflow/stream_executor/cuda/cuda_diagnostics.c

In [38]:
reload(rl_dqn)
reload(environment)
reload(embedding)

from rl_dqn import ReplayMemory, DQN, Transition
from embedding import EmbeddingModel
from environment import OriginAgent, DialougeEnv

In [4]:
embedding_model = EmbeddingModel()
unmasker = pipeline('fill-mask', model='xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
origin_agent = OriginAgent()
env = environment.DialougeEnv(origin_agent,embedding_model,unmasker)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# go to school

In [8]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0

def select_action(state,eps_threshold=None):
    global steps_done
    sample = random.random()
    if  eps_threshold is not None:
        eps_threshold = 0
    else:
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

num_episodes = 3000
return_list = []
episode_durations = []


for i_episode in tqdm.tqdm(range(num_episodes)):
    # Initialize the environment and get it's state
    returns = 0
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        returns += reward
        done = terminated or truncated
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        # Move to the next state
        state = next_state
        # Perform one step of the optimization (on the policy network)
        optimize_model()
        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        if done:
            return_list.append(returns)
            # plot_durations()
            break

turn = 0
for his in env.history:
    space_num = 0 if turn%2==0 else (135-len(his.text))
    print(turn, space_num) 
    print(' '*space_num + his.text)
    print('----------------------------------')
    turn += 1

  0%|                                                                                                              | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_575/2957816014.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
/tmp/ipykernel_575/2957816014.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
  0%|▏                                                                                                   | 5/3000 [00:11<1:51:42,  2.24s/it]


KeyboardInterrupt: 

In [12]:
state,info =  env.reset()
print(env.history[0])
env.get_option_words_by_llm()

{'text': 'U.S. Representative Tony Hall arrived in Jordan on\n\t\t\t\tSaturday en route to Iraq, where he is expected to look into the plight\n\t\t\t\tof Iraqis after nearly 10 years of U.N. trade sanctions.\n\t\t\tHall, an Ohio Democrat and one of very few U.S. congressmen to visit\n\t\t\t\tIraq since the 1991 Gulf War over Kuwait, is scheduled to embark Sunday\n\t\t\t\tthe 12-hour overland trip to the Iraqi capital, Baghdad.\n\t\t\tHe did not speak to reporters in Jordan, but he told The Associated\n\t\t\t\tPress before leaving the United States that he hopes to "separate the\n\t\t\t\thumanitarian work from the political issues."', 'target': 'trip', 'substitutes': [('journey', 0.8), ('voyage', 0.5), ('trek', 0.5), ('excursion', 0.5), ('visit', 0.4), ('expedition', 0.4), ('tour', 0.3), ('outing', 0.3), ('travel', 0.2), ('errand', 0.1), ('peregrination', 0.1), ('quest', 0.1), ('swing', 0.0), ('indiscretion', 0.0), ('mistake', 0.0), ('ramble', 0.0), ('cruise', 0.0), ('blunder', 0.0), ('

[('trip', 0.455),
 ('journey', 0.222),
 ('visit', 0.114),
 ('trek', 0.089),
 ('voyage', 0.089)]

## Expierment

In [4]:
state,info =  env.reset()

In [18]:
env.history[0]

question(text='I imagine that people do not come here for unimportant reasons.”\n            It hissed thoughtfully. “True, true.', target='hissed', substitutes=[('seethe', 0.4), ('jeer', 0.3), ('make buzzing sound', 0.3), ('whirr', 0.3), ('whistle', 0.2), ('sputter', 0.2), ('mock', 0.2), ('shrill', 0.1), ('boo', 0.1), ('deride', 0.0), ('blow', 0.0), ('whiz', 0.0), ('rasp', 0.0), ('hoot', 0.0), ('say', 0.0), ('wheeze', 0.0), ('disapprove', 0.0), ('damn', 0.0), ('sizzle', 0.0), ('catcall', 0.0), ('buzz', 0.0), ('sibilate', 0.0), ('revile', 0.0), ('ridicule', 0.0), ('spit', 0.0), ('sigh', 0.0), ('condemn', 0.0), ('shout down', 0.0), ('whisper', 0.0), ('decry', 0.0), ('siss', 0.0)])

In [ ]:
self = env

In [514]:
h0 = self.history[0]

mask_show = ''
solo_mask = h0.text.replace(h0.target,'<mask>',1)
mask_text = ''
for i,(word,score) in enumerate(h0.substitutes[:5]):
    if len(h0.text)*6<2100:
        text = h0.text
    else:
        subtract_len = len(h0.text)-350
        index = h0.text.index(h0.target)
        if i%2==0:
            pre_sub_index = min(0+subtract_len,index-20)
            text = h0.text[pre_sub_index:]
        else:
            post_sub_index = max(len(h0.text)-subtract_len,index+20)
            text = h0.text[:post_sub_index]
    text.replace(h0.target,word,1)
    if len(self.model.tokenizer(mask_text+text+solo_mask)['input_ids'])>512:
        break
    mask_text += text.replace(h0.target,word,1)
    # mask_show += text.replace(h0.target,f"\033[31m{word}\033[0m",1)+'\033[31m<\nnewline>\033[0m'

In [ ]:
#

[('even', 0.913),
 ('and', 0.022),
 ('particularly', 0.007),
 ('especially', 0.003),
 ('although', 0.003)]

In [546]:
state,info = env.reset()

In [547]:
env.history

[question(text='In its center, under a spearing, white light, was a golden table draped with a blue velvet cloth, on which lay a gray, plum-sized rock.\n            “A marvel, isn’t it?” she said.', target='marvel', substitutes=[('wonder', 0.9), ('phenomenon', 0.6), ('miracle', 0.5), ('amazement', 0.5), ('awed', 0.2), ('be amazed be', 0.1), ('genius', 0.0), ('goggle', 0.0), ('prodigy', 0.0), ('gaze', 0.0), ('feel surprise', 0.0), ('stare', 0.0), ('stand in awe', 0.0), ('be surprised', 0.0), ('sensation', 0.0), ('gape', 0.0)])]

In [548]:
env.get_option_words_by_llm()

[('wonder', 0.686),
 ('miracle', 0.053),
 ('magic', 0.03),
 ('strange', 0.012),
 ('wonderful', 0.01)]

In [465]:
from environment import Action

##  evaluate

In [531]:
reload(environment)

<module 'environment' from '/mnt/d/BaiduSyncdisk/intelligent_interactive_system/Thesis/RL/environment.py'>

In [40]:
test_agent = environment.OriginAgent('../data/swords/swords-v1.1_test.json.gz')
test_env = environment.DialougeEnv(test_agent,embedding_model,unmasker)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
context_id_state_mapping = {}
context_option_mapping = {}
for context_id in tqdm.tqdm(env.OA.context_ids):
    state,info = env.reset(context_id)
    option_words = env.get_option_words_by_llm(context_id)
    context_id_state_mapping[context_id] = state
    context_option_mapping[context_id] = option_words

 35%|███████████████████████████████████▏                                                                 | 129/370 [01:42<03:12,  1.25it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 327002616 bytes. Error code 12 (Cannot allocate memory)

In [2]:
context_id_state_mapping

NameError: name 'context_id_state_mapping' is not defined

In [29]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import spacy

nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


In [36]:
def get_option_words_by_llm(self,context_id):
    # state, info = test_env.reset()
    # h0 =self.history[0]


    def repeat_part(sent,target,substitutes,trunck=False):
        rep_list = []
        substitutes = [w for w,s in substitutes[:5]]
        for sub in [target]+substitutes+['<mask>']:
            start = offset-30 if trunck else 0
            repeat_part = f"{sent.text[start:offset-sent.start_char]}{sub}{sent.text[offset-sent.start_char+len(target):]}"
            rep_list.append(repeat_part)
        return '.'.join(rep_list)

    h0,_ = self.OA.sample(context_id)
    # h0 = self.history[0]

    offset = h0['offset']
    mask_sentence_list = []
    for sent in nlp(h0['text']).sents:
        print(sent.start_char, offset ,sent.end_char)
        if sent.start_char <= offset <sent.end_char:
            sent_text = repeat_part(sent, h0['target'], h0['substitutes'])
            mask_sentence_list.append(sent_text)
        else:
            mask_sentence_list.append(sent.text)

    mask_text = ''.join(mask_sentence_list)
    token_lens = len(tokenizer(''.join(mask_text))['input_ids'])

    if token_lens>512:
        mask_sentence_list = []
        for sent in nlp(h0['text']).sents:
            if sent.start_char <= offset <sent.end_char:
                sent_text = repeat_part(sent, h0['target'], h0['substitutes'],True)
                # print('-------------')
                mask_sentence_list.append(sent_text)
            else:
                mask_sentence_list.append(sent.text)
        mask_text = ''.join(mask_sentence_list)
        token_lens = len(tokenizer(''.join(mask_text))['input_ids'])                
    # words = [(token['token_str'],round(token['score'],3)) for token in self.mask_model(mask_text)]
    return mask_text


In [33]:
h0 = env.history[0]
h0['text'][h0['offset']:]

'Let’s go.”\n            As we headed down the sidewalk, I said, “What is your name, anyway?”'

In [37]:
get_option_words_by_llm(env,context_id)

0 20 19
20 20 43
43 20 111


'“You’ve got a deal.Let’s go.”\n            .off we’s go.”\n            .allow’s go.”\n            .we will’s go.”\n            .enable’s go.”\n            .live with’s go.”\n            .<mask>’s go.”\n            As we headed down the sidewalk, I said, “What is your name, anyway?”'

In [18]:
env.OA.sample()

({'text': 'It was last February, after the winter break, that we moved in together. Now spring was back, under the concrete, and I could smell it even here.\n            “Isn’t it an amazing night, Rache?',
  'target': 'even',
  'substitutes': [('still', 0.4),
   ('as well as', 0.2),
   ('so much as', 0.1),
   ('much', 0.1),
   ('in spite of', 0.0),
   ('yet', 0.0),
   ('despite', 0.0),
   ('notwithstanding', 0.0),
   ('indeed', 0.0),
   ('actually', 0.0),
   ('disregarding', 0.0),
   ('more', 0.0),
   ('yet all the', 0.0)],
  'offset': 135},
 'c:8449d1484b624c8db76ae3d9c60a000f677a244d')

In [536]:
returns = 0
action_list = []
turns_list = []
reward_list = []
for i in tqdm.tqdm(range(200)):
    context_id = test_env.OA.context_ids[i]
    state, info = test_env.reset(context_id)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    episode_returns = 0
    for t in count():
        action = select_action(state,eps_threshold=None)
        action_list.append(action.item())
        observation, reward, terminated, truncated, _ = test_env.step(action.item())
        state = observation.reshape((1,-1))
        reward = torch.tensor([reward], device=device)
        episode_returns += reward
        # returns += reward
        done = terminated or truncated
        
        if done:
            turns_list.append(t)
            reward_list.append(episode_returns)
            # plot_durations()


            turn = 0
            for his in env.history:
                space_num = 0 if turn%2==0 else (135-len(his.text))
                role = "User:" if  turn%2==0 else  "Bot:"
                # print(role)
                # print(his.text)
                # print('----------------------------------')
                turn += 1
            break

    # break

  0%|                                                                                   | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_142/1917932590.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
100%|█████████████████████████████████████████████████████████████████████████| 200/200 [18:37<00:00,  5.59s/it]


In [538]:
from collections import Counter
import pandas as pd

In [539]:
Counter(action_list)

Counter({2: 919, 1: 16, 3: 21, 0: 13})

In [540]:
import pandas as pd

In [541]:
data_act = Counter(action_list) # = 
name_map = {0:'no action',1:'confirm',2:'options',3:'more info'}
pd.DataFrame([data_act],index=['number of occurance']).rename(name_map,axis=1).T

,number of occurance
options,919
confirm,16
more info,21
no action,13


In [542]:
df2 = pd.DataFrame([Counter(turns_list)]).T.reset_index()
df2.columns = ['turns','Number of Occurrence']
df2['turns'] += 1
df2

,turns,Number of Occurrence
0,5,188
1,2,4
2,3,2
3,1,3
4,4,3


In [135]:
eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)

In [214]:
eps_threshold

0.4036247594865046

In [ ]:
self = origin_agent

In [54]:
context_id = random.choice(self.context_ids)
context = self.contexts[context_id]['context']
target = self.contexts[context_id]['targets'][0]
target_text = target['target']
substitutes = [(sub['substitute'],sub['label_score']) for sub in target['substitutes']]
sorted_subs = sorted(substitutes,key=lambda x:x[1],reverse=True)

In [113]:
import numpy as np

In [43]:
return_list

[tensor([-0.1000])]

In [42]:
average_return = []
len_avg = 10
for i in range(len(return_list)-len_avg):
    average_return.append(np.mean([s.item() for s in return_list[i:i + len_avg]]))

In [40]:
np.mean([s.item() for s in return_list[i:i+10]])

NameError: name 'np' is not defined

In [19]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [20]:
import spacy

nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


In [113]:
def repeat_part(sent,target,substitutes,trunck=False):
    rep_list = []
    substitutes = [w for w,s in substitutes[:5]]
    for sub in [target]+substitutes+['<mask>']:
        start = offset-30 if trunck else 0
        repeat_part = f"{sent.text[start:offset-sent.start_char]}{sub}{sent.text[offset-sent.start_char+len(target):]}"
        rep_list.append(repeat_part)
    return '.'.join(rep_list)

In [94]:
import tqdm

In [98]:
test_origin_agent = OriginAgent(file_path='../data/swords/swords-v1.1_test.json.gz')
test_env = environment.DialougeEnv(test_origin_agent,embedding_model,unmasker)

In [114]:
for con in tqdm.tqdm(env.OA.context_ids):
    state,info = env.reset(con)
    h0 = env.history[0]
    offset = h0['offset']
    mask_sentence_list = []
    for sent in nlp(h0['text']).sents:
        if sent.start_char < offset <sent.end_char:
            sent_text = repeat_part(sent, h0['target'], h0['substitutes'])
            # print('-------------')
            mask_sentence_list.append(sent_text)
        else:
            mask_sentence_list.append(sent.text)

    mask_text = ''.join(mask_sentence_list)
    token_lens = len(tokenizer(''.join(mask_text))['input_ids'])
    token_lens, mask_text
    
    if token_lens>512:
        mask_sentence_list = []
        for sent in nlp(h0['text']).sents:
            if sent.start_char < offset <sent.end_char:
                sent_text = repeat_part(sent, h0['target'], h0['substitutes'],True)
                # print('-------------')
                mask_sentence_list.append(sent_text)
            else:
                mask_sentence_list.append(sent.text)
        mask_text = ''.join(mask_sentence_list)
        token_lens = len(tokenizer(''.join(mask_text))['input_ids'])
        if token_lens>512:
            print(token_lens)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 370/370 [00:34<00:00, 10.59it/s]


In [111]:
for sent in nlp(h0['text']).sents:
    print("-----------")
    print(sent)
    print(sent)
# print(mask_text)

-----------
The FBI cited a vivid conversation with Anissina's mother in
which Tokhtakhounov assured her that even if her daughter "falls,
we will make sure she is No. 1."


   After the Winter Olympics, a French judge, Marie-Reine Le
Gougne, was suspended by the International Skating Union for not
reporting pressure she said was put on her by Didier Gailhaguet,
president of the French Skating Federation, to vote for the Russian
pairs team.


   
-----------
She later recanted and said that Canadian officials had
pressured her.


In [53]:
sent.text[]

'B'

In [28]:
for i,t in enumerate(text[0:19]):
    print(i,t,end=';')

0 ";1 W;2 e;3  ;4 w;5 e;6 r;7 e;8  ;9 a;10 l;11 l;12  ;13 a;14 p;15 p;16 a;17 l;18 l;

In [37]:
sent.start_char

117

In [65]:
sent.as_doc

<function Span.as_doc>

In [35]:
origin_agent

In [35]:
import spacy

# 加载英文语言模型
nlp = spacy.load("en_core_web_sm")

In [ ]:
def truncat(text):
    text_list = []
    
    
    

In [103]:
state,info = env.reset()

In [106]:
h0 = env.history[0]
mask_text_doc = nlp(h0.text)
mask_text = ''
# 遍历每个句子并打印
for sentence in mask_text_doc.sents:
    # print(sentence.text)
    if h0.target in sentence.text:
        words = [h0.tarnlp = spacy.load("en_core_web_sm")get,*[w for w,score in h0.substitutes[:5]],'<mask>']
        sub_sentence_text = ".".join([sentence.text.replace(h0.target,w,1) for w in words])
        mask_text += sub_sentence_text
    else:
        mask_text += sentence.text
if len(tokenizer(mask_text)['input_ids'])>10:
    mask_text = ''
    # 遍历每个句子并打印
    for sentence in mask_text_doc.sents:
        # print(sentence.text)
        if h0.target in sentence.text:
            parts = sentence.text.split(",")
            mask_idx = [i for i,p in enumerate(parts) if (h0.target in p)][0]
            words = [h0.target,*[w for w,score in h0.substitutes[:7]],'<mask>']
            sub_sentence_text = ",".join([parts[mask_idx].replace(h0.target,w,1) for w in words])
            parts[mask_idx] = sub_sentence_text
            mask_text += ' '.join(parts)
        else:
            mask_text += sentence.text

print(h0.text)
print("-------------------------------------")
print(mask_text)
print(f"-----------{h0.target}--------------")
print(h0.substitutes)
print("-------------------------")
print([(token['token_str'],round(token['score'],3)) for token in unmasker(mask_text)])
print("-------------------------")
print([(token['token_str'],round(token['score'],3)) for token in unmasker(solo_mask)])

len(tokenizer(h0.text)['input_ids'])

list>   

Words of wisdom


   Julia Child doesn't have much use for fads and trends -- never
has.
-------------------------------------
list>   

Words of wisdom


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of the wise


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of knowledge


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of advice


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of astuteness


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of sageness


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of enlightenment


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of expertise


   Julia Child doesn't have much use for fads and trends -- never
has.,list>   

Words of <mask>


   Julia Child doesn't

26

In [107]:
h0 = env.history[0]
mask_text_doc = nlp(h0.text)
mask_text = ''
# 遍历每个句子并打印
for sentence in mask_text_doc.sents:
    # print(sentence.text)
    if h0.target in sentence.text:
        words = [h0.target,*[w for w,score in h0.substitutes[:7]],'<mask>']
        random.shuffle(words)
        sub_sentence_text = ".".join([sentence.text.replace(h0.target,w,1) for w in words])
        mask_text += sub_sentence_text
    else:
        mask_text += sentence.text
if len(tokenizer(mask_text)['input_ids'])>512:
    mask_text = ''
    # 遍历每个句子并打印
    for sentence in mask_text_doc.sents:
        # print(sentence.text)
        if h0.target in sentence.text:
            parts = sentence.text.split(",")
            mask_idx = [i for i,p in enumerate(parts) if (h0.target in p)][0]
            words = [h0.target,*[w for w,score in h0.substitutes[:5]],'<mask>']
            random.shuffle(words)
            sub_sentence_text = ",".join([parts[mask_idx].replace(h0.target,w,1) for w in words])
            parts[mask_idx] = sub_sentence_text
            mask_text += ' '.join(parts)
        else:
            mask_text += sentence.text

print(h0.text)
print("-------------------------------------")
print(mask_text)
print(f"-----------{h0.target}--------------")
print(h0.substitutes)
print("-------------------------")
print([(token['token_str'],round(token['score'],3)) for token in unmasker(mask_text)])
print("-------------------------")
print([(token['token_str'],round(token['score'],3)) for token in unmasker(solo_mask)])

len(tokenizer(h0.text)['input_ids'])

list>   

Words of wisdom


   Julia Child doesn't have much use for fads and trends -- never
has.
-------------------------------------
list>   

Words of sageness


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of wisdom


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of knowledge


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of astuteness


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of <mask>


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of advice


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of the wise


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of expertise


   Julia Child doesn't have much use for fads and trends -- never
has..list>   

Words of enlightenment


   Julia Child doesn't

26

In [97]:
import random
random.shuffle(words)

In [98]:
words

['listless', '<mask>', 'lifeless', 'empty', 'dead', 'flat', 'deathly']

In [45]:
len(tokenizer(mask_text)['input_ids'])>512

True

In [ ]:
if there is target in the top masks :
    that means LLM could predict well, the ambiagrous is less
    
    No action

if top mask's score is high ,and in':
    
    

In [260]:
l0s = 0
l1s=0
for i in  range(100):
    state,info = test_env.reset()
    h0 = test_env.history[0]
    l1 = len(tokenizer(h0.text)['input_ids'])
    l0 = len(h0.text)
    l0s+=l0
    l1s+=l1
    print(l0,l1,round(l0/l1,2))

117 31 3.77
276 77 3.58
142 41 3.46
404 106 3.81
258 70 3.69
129 36 3.58
89 20 4.45
358 96 3.73
294 61 4.82
313 81 3.86
426 119 3.58
213 41 5.2
292 65 4.49
152 29 5.24
204 62 3.29
251 59 4.25
425 106 4.01
266 65 4.09
280 61 4.59
321 67 4.79
65 16 4.06
375 84 4.46
546 116 4.71
194 45 4.31
148 30 4.93
183 42 4.36
134 31 4.32
207 48 4.31
510 105 4.86
297 58 5.12
277 64 4.33
242 53 4.57
219 46 4.76
243 59 4.12
369 87 4.24
420 76 5.53
226 60 3.77
296 73 4.05
131 35 3.74
111 26 4.27
192 45 4.27
164 42 3.9
93 28 3.32
446 106 4.21
336 96 3.5
420 76 5.53
245 61 4.02
258 70 3.69
290 64 4.53
154 34 4.53
293 69 4.25
197 52 3.79
224 58 3.86
195 44 4.43
153 35 4.37
112 27 4.15
129 36 3.58
386 93 4.15
226 47 4.81
270 69 3.91
343 77 4.45
422 113 3.73
171 45 3.8
540 134 4.03
335 73 4.59
259 63 4.11
293 69 4.25
103 27 3.81
185 43 4.3
153 35 4.37
325 79 4.11
277 65 4.26
73 20 3.65
433 109 3.97
245 61 4.02
354 74 4.78
129 36 3.58
376 98 3.84
232 58 4.0
595 132 4.51
267 52 5.13
475 96 4.95
154 34 4.53
201 

In [371]:
state,info = test_env.reset()
h0 = test_env.history[0]
solo_mask = h0.text.replace(h0.target,'<mask>',1)
mask_text = ''
for i,(word,score) in enumerate(h0.substitutes[:5]):
    print(len(h0.text)*6)
    if len(h0.text)*6<2100:
        text = h0.text
    else:
        subtract_len = len(h0.text)-350
        index = h0.text.index(h0.target)
        post_sub_index = max(len(h0.text)-subtract_len,index+20)
        if i%2==0:
            pre_sub_index = min(0+subtract_len,index-20)
            text = h0.text[pre_sub_index:]
    if len(tokenizer(mask_text+text+solo_mask)['input_ids'])>512:
        break
    mask_text += text
mask_text  += solo_mask
    

2502
2502
2502
2502
2502


In [373]:
mask_text

"            compromise and capitulation, between symbols and substance. Shortly before noon,\n                    I climbed the sandy path to Dream's End, rehearsing my apology, slowing only to\n                    pluck burrs from between my sandals and my feet. \n                 Malaquez answered the door in blood-red pajamas and a black silk robe.            compromise and capitulation, between symbols and substance. Shortly before noon,\n                    I climbed the sandy path to Dream's End, rehearsing my apology, slowing only to\n                    pluck burrs from between my sandals and my feet. \n                 Malaquez answered the door in blood-red pajamas and a black silk robe.            compromise and capitulation, between symbols and substance. Shortly before noon,\n                    I climbed the sandy path to Dream's End, rehearsing my apology, slowing only to\n                    pluck burrs from between my sandals and my feet. \n                 Malaquez a

In [368]:
len(mask_text)

2084

In [266]:
import random

In [280]:
random.randint(0,1)

0

In [281]:
text = 'reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move '

In [359]:
m_text = (text*12)[:-23]+ '<mask>'

In [360]:
token = tokenizer(m_text)

In [361]:
len(token['input_ids'])

512

In [362]:
unmasker(m_text)

[{'score': 0.8441550135612488,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for do-it-yourself analysis.,Also spurring the move reudToy, a pillow bearing the likeness of Sigmund Freud, is marketed as a $24.95 tool for

In [264]:
512*4.1

2099.2

In [265]:
2100/6

350.0

In [261]:
(l0s/l1s)

4.199420569773056

In [251]:
len(mask_text)

1559

In [250]:
h0.text.index(h0.target),len(h0.text)

(157, 387)

In [247]:
512 - len(h0.text) 

125

In [154]:
from functools import lru_cache

In [85]:
import transformers

In [ ]:
transformers.AutoModelForMaskedLM(

In [7]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [88]:
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

In [93]:
# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input)

In [101]:
output

MaskedLMOutput(loss=None, logits=tensor([[[ 6.4861e+01,  1.6882e-02,  3.7656e+01,  ...,  2.1584e+01,
           1.4380e+01,  1.8790e+01],
         [ 2.7493e+01, -1.4091e+00,  6.4847e+01,  ...,  4.0234e+01,
           1.6296e+01,  3.0925e+01],
         [ 1.9604e+01, -1.2597e+00,  4.8981e+01,  ...,  3.5830e+01,
           1.7145e+01,  2.7173e+01],
         ...,
         [ 2.2920e+01, -1.4657e+00,  5.1211e+01,  ...,  3.8495e+01,
           1.6508e+01,  2.7687e+01],
         [ 2.8598e+01, -1.2868e+00,  6.7706e+01,  ...,  4.4857e+01,
           1.8004e+01,  3.5004e+01],
         [ 4.4955e+01, -2.1554e-01,  4.9643e+01,  ...,  2.8253e+01,
           1.6841e+01,  2.3610e+01]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [104]:
model.__class__

transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForMaskedLM

In [ ]:
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits


In [105]:
from transformers import FillMaskPipeline

2023-05-17 16:07:54.349083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-17 16:07:54.349119: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [108]:
mask_pipeline = FillMaskPipeline(model=env.model,tokenizer=tokenizer)

AttributeError: 'EmbeddingModel' object has no attribute 'config'

In [107]:
mask_pipeline("this is a <mask> car")

[{'score': 0.07687385380268097,
  'token': 34923,
  'token_str': 'beautiful',
  'sequence': 'this is a beautiful car'},
 {'score': 0.04615773260593414,
  'token': 29681,
  'token_str': 'sports',
  'sequence': 'this is a sports car'},
 {'score': 0.03222648799419403,
  'token': 54704,
  'token_str': 'classic',
  'sequence': 'this is a classic car'},
 {'score': 0.031460631638765335,
  'token': 6782,
  'token_str': 'great',
  'sequence': 'this is a great car'},
 {'score': 0.030774081125855446,
  'token': 26267,
  'token_str': 'nice',
  'sequence': 'this is a nice car'}]

In [118]:
embedding_model.model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0): XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
     

In [102]:
model

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
          